# Smoking is bad

In [ ]:
# TODO: install all the necessary packages
!pip install pandas
!pip install matplotlib

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
# TODO: load the data using the pathlib library to create the path to your data
data_path = Path('data') / 'smoking_data.csv'
df = pd.read_csv(data_path)

In [ ]:
# TODO: what are the dimensions of the data frame?
np.shape(df)
df.columns

In [ ]:
# TODO: use .describe() on your data frame: what do you notice about the string columns?
df.describe()

the string columns are not included in the describe function --- unnamed column is the individual count number i think (i.e. individual 0 is in col2)

In [ ]:
# TODO: do not forget to check the null values

df.info()

df.isnull().sum()

700 null values for salary -- all other columns have no null values 
- do we drop or imput? 
for now let us drop the the 700 rows with null values for salary and so we then have 614 individuals (lowers the n drastically but alas)

In [ ]:
# TODO: prepare your data frame so that you can work with
# Create a copy to preserve original data
df_prepared = df.copy()

# Drop rows with null values in salary column
df_prepared = df_prepared.dropna(subset=['salary'])


# Convert categorical string columns to boolean
df_prepared['smoker'] = np.where(df_prepared['smoker'] == "Yes", True, False)
df_prepared['alive'] = np.where(df_prepared['outcome'] == "Alive", True, False)
df_prepared['female'] = np.where(df_prepared['gender'] == "female", True, False)

# Drop the original categorical columns (outcome and gender) since we've converted them
df_prepared = df_prepared.drop(['outcome', 'gender'], axis=1)

# Drop the 'Unnamed: 0' column as it's just an index
df_prepared = df_prepared.drop(['Unnamed: 0'], axis=1)

# Reset index after dropping rows
df_prepared = df_prepared.reset_index(drop=True)

# Verify the preparation
print(f"Original shape: {df.shape}")
print(f"Prepared shape: {df_prepared.shape}")
print(f"\nPrepared DataFrame info:")
print(df_prepared.info())
print(f"\nFirst few rows:")
print(df_prepared.head())

In [ ]:
df_prepared['female'].unique()

## Smoking analysis
Let's first check what your cousin did.

In [ ]:
# TODO: Reimplement the small code snippet described in the problem set instructions.

df_prepared.groupby(["smoker"]).agg(prob=("alive", np.mean))

smokes  | prob 
0       | 0.686
1       | 0.761 

above is the results given in PS

- note the numbers are not exxactlt the same but that is probably because i dropped all of the null values ( i have verified, without dropping the code gives the same values )

You should get the same results.

# Is smoking really good?
Now it is up to you to run the analysis further and check what could be underlying your cousings results.

first we want to test if there's any bias regarding age: 
intuition: if smokers are, on average younger, and younger people are less likely to die, we may think that the cousin's results are instead showing the impact of age on survival rather than smoking 

hypothesis: There is no significant age difference between smokers and non-smokers in this dataset.

In [ ]:
print("### TEST 1: ARE SMOKERS YOUNGER? ###")
print()

# Descriptive statistics
age_by_smoking = df_prepared.groupby('smoker')['age'].describe()
age_by_smoking.index = ['Non-Smoker', 'Smoker']
print(age_by_smoking[['count', 'mean', 'std', 'min', 'max']])
print()

smoker_age_mean = df_prepared[df_prepared['smoker'] == True]['age'].mean()
nonsmoker_age_mean = df_prepared[df_prepared['smoker'] == False]['age'].mean()
age_difference = smoker_age_mean - nonsmoker_age_mean

print(f"Mean age of smokers: {smoker_age_mean:.2f} years")
print(f"Mean age of non-smokers: {nonsmoker_age_mean:.2f} years")
print(f"Difference: {age_difference:.2f} years")
print()

In [ ]:
# Statistical test for age difference
from scipy import stats

smoker_ages = df_prepared[df_prepared['smoker'] == True]['age']
nonsmoker_ages = df_prepared[df_prepared['smoker'] == False]['age']


In [ ]:
# Two-sample t-test
t_stat, p_value = stats.ttest_ind(smoker_ages, nonsmoker_ages)

print(f"Independent t-test results:")
print(f"  H₀: μ_smoker = μ_nonsmoker")
print(f"  H₁: μ_smoker ≠ μ_nonsmoker")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.6f}")
print(f"  Significance level: α = 0.05")
print()

if p_value < 0.05:
    print(f"✓ RESULT: REJECT H₀ (p < 0.05)")
    print(f"  There IS a statistically significant age difference")
    if age_difference < 0:
        print(f"  Smokers are younger by {abs(age_difference):.2f} years on average")
    else:
        print(f"  Smokers are older by {abs(age_difference):.2f} years on average")
else:
    print(f"✗ RESULT: FAIL TO REJECT H₀ (p ≥ 0.05)")
    print(f"  No statistically significant age difference detected")
print()
print("-" * 70)
print()


the difference of smokers being 4.41 years younger is statistically significant --- which supports the hypothesis that perhaps the cousin's finding is impacted by the age dynamic 

to build on this further we may consider running an logistic regression estimator to see how much each of the factors impact the probability of being alive 
-- use logistic regression since the outcome varaible is binary (alive: true; false)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

print("=" * 70)
print("LOGISTIC REGRESSION: Predicting Survival")
print("=" * 70)
print()

# Prepare the data for regression
# Features: age, salary, smoker, female
# Target: alive

X = df_prepared[['age', 'salary', 'smoker', 'female']].copy()
y = df_prepared['alive'].copy()

# Convert boolean to int for regression
X['smoker'] = X['smoker'].astype(int)
X['female'] = X['female'].astype(int)

print(f"Sample size: {len(X)} observations")
print(f"Features: {list(X.columns)}")
print(f"Target: alive (1=Alive, 0=Dead)")
print()

# =====================
# MODEL 1: Without Standardization (for interpretable coefficients)
# =====================
print("### MODEL 1: LOGISTIC REGRESSION (Raw Coefficients) ###")
print()

# Fit logistic regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X, y)

# Get coefficients and intercept
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_reg.coef_[0],
    'Odds Ratio': np.exp(log_reg.coef_[0])
})
coefficients = coefficients.sort_values('Coefficient', key=abs, ascending=False)

print(coefficients.to_string(index=False))
print()
print(f"Intercept: {log_reg.intercept_[0]:.4f}")
print()

# Interpretation
print("INTERPRETATION:")
print("-" * 70)
for _, row in coefficients.iterrows():
    feature = row['Feature']
    coef = row['Coefficient']
    odds_ratio = row['Odds Ratio']
    
    if feature == 'age':
        print(f"\n{feature.upper()}:")
        print(f"  Coefficient: {coef:.6f}")
        print(f"  Odds Ratio: {odds_ratio:.6f}")
        print(f"  → Each additional year of age multiplies odds of survival by {odds_ratio:.4f}")
        print(f"  → Each 10 years multiplies odds by {odds_ratio**10:.4f}")
        if coef < 0:
            print(f"  → NEGATIVE effect: Older people less likely to survive")
        else:
            print(f"  → POSITIVE effect: Older people more likely to survive")
            
    elif feature == 'smoker':
        print(f"\n{feature.upper()}:")
        print(f"  Coefficient: {coef:.6f}")
        print(f"  Odds Ratio: {odds_ratio:.6f}")
        print(f"  → Being a smoker multiplies odds of survival by {odds_ratio:.4f}")
        pct_change = (odds_ratio - 1) * 100
        if coef < 0:
            print(f"  → NEGATIVE effect: Smokers have {abs(pct_change):.1f}% LOWER odds of survival")
        else:
            print(f"  → POSITIVE effect: Smokers have {pct_change:.1f}% HIGHER odds of survival")
            
    elif feature == 'female':
        print(f"\n{feature.upper()}:")
        print(f"  Coefficient: {coef:.6f}")
        print(f"  Odds Ratio: {odds_ratio:.6f}")
        print(f"  → Being female multiplies odds of survival by {odds_ratio:.4f}")
        pct_change = (odds_ratio - 1) * 100
        if coef < 0:
            print(f"  → NEGATIVE effect: Females have {abs(pct_change):.1f}% lower odds")
        else:
            print(f"  → POSITIVE effect: Females have {pct_change:.1f}% higher odds")
            
    elif feature == 'salary':
        print(f"\n{feature.upper()}:")
        print(f"  Coefficient: {coef:.9f}")
        print(f"  Odds Ratio (per $1): {odds_ratio:.9f}")
        odds_ratio_10k = odds_ratio ** 10000
        print(f"  → Each $10,000 increase multiplies odds by {odds_ratio_10k:.6f}")
        pct_change = (odds_ratio_10k - 1) * 100
        if coef < 0:
            print(f"  → NEGATIVE effect: Each $10k reduces odds by {abs(pct_change):.2f}%")
        else:
            print(f"  → POSITIVE effect: Each $10k increases odds by {pct_change:.2f}%")

print()
print("-" * 70)
print()

# Model performance
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = log_reg.predict(X)
accuracy = accuracy_score(y, y_pred)

print(f"Model Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print()

print("Confusion Matrix:")
cm = confusion_matrix(y, y_pred)
cm_df = pd.DataFrame(cm, 
                     index=['Actually Dead', 'Actually Alive'],
                     columns=['Predicted Dead', 'Predicted Alive'])
print(cm_df)
print()

# =====================
# MODEL 2: With Standardized Features (for comparison)
# =====================
print("### MODEL 2: STANDARDIZED LOGISTIC REGRESSION ###")
print("(For comparing relative importance of features)")
print()

# Standardize continuous variables only
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[['age', 'salary']] = scaler.fit_transform(X[['age', 'salary']])

# Fit model
log_reg_scaled = LogisticRegression(random_state=42, max_iter=1000)
log_reg_scaled.fit(X_scaled, y)

# Get standardized coefficients
coefficients_scaled = pd.DataFrame({
    'Feature': X.columns,
    'Standardized Coefficient': log_reg_scaled.coef_[0],
    'Absolute Magnitude': np.abs(log_reg_scaled.coef_[0])
})
coefficients_scaled = coefficients_scaled.sort_values('Absolute Magnitude', ascending=False)

print(coefficients_scaled[['Feature', 'Standardized Coefficient']].to_string(index=False))
print()

print("RELATIVE IMPORTANCE (by absolute magnitude):")
for idx, row in coefficients_scaled.iterrows():
    print(f"  {row['Feature']:10s}: |{row['Standardized Coefficient']:7.4f}|")
print()

# =====================
# Statistical Significance (using statsmodels for p-values)
# =====================
print("### MODEL 3: STATISTICAL SIGNIFICANCE ###")
print()

import statsmodels.api as sm

# Add constant for statsmodels
X_with_const = sm.add_constant(X)

# Fit logistic regression with statsmodels
logit_model = sm.Logit(y, X_with_const)
result = logit_model.fit(disp=0)

print(result.summary2().tables[1])
print()

print("SIGNIFICANCE INTERPRETATION (α = 0.05):")
for var in X.columns:
    p_val = result.pvalues[var]
    coef = result.params[var]
    if p_val < 0.05:
        print(f"  {var:10s}: p={p_val:.4f} → SIGNIFICANT {'(+)' if coef > 0 else '(-)'}")
    else:
        print(f"  {var:10s}: p={p_val:.4f} → NOT significant")
print()

# =====================
# KEY FINDINGS
# =====================
print("=" * 70)
print("KEY FINDINGS:")
print("=" * 70)

smoker_coef = log_reg.coef_[0][list(X.columns).index('smoker')]
age_coef = log_reg.coef_[0][list(X.columns).index('age')]

if smoker_coef > 0:
    print("\n⚠️  WARNING: Smoker coefficient is POSITIVE!")
    print("   This suggests smokers have better survival odds.")
    print("   BUT this is misleading because:")
    print(f"   - Age coefficient is {age_coef:.6f} (younger = better survival)")
    print("   - Smokers are younger on average")
    print("   - The positive smoker effect is capturing the age effect")
    print("\n   CONCLUSION: The model shows confounding, not causation!")
elif smoker_coef < 0:
    print(f"\n✓ Smoker coefficient is NEGATIVE ({smoker_coef:.6f})")
    print("   This correctly shows smoking reduces survival odds")
    print("   even after controlling for age, gender, and salary.")
else:
    print("\n  Smoker coefficient is approximately zero")
    print("  Smoking has no significant effect after controlling for other factors")

print("\n" + "=" * 70)

positive coefficients on: female -- being female increases survival likelihood 

negative coefficients on: smoker and age 

-  thereby implying smoking decreases p(alive)
similarly age decreases P(alive)

- given our original analysis showed that smokers were on average younger this may demonstrate why our cousin's findings were misleading 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'

# Data from your logistic regression
features = ['Female', 'Smoker', 'Age', 'Salary']
coefficients = [0.714646, -0.242709, -0.126916, -0.000001]
odds_ratios = [2.043464, 0.784499, 0.880808, 0.999999]
percent_changes = [104.3, -21.6, -11.9, 0.0]

# Create figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# ============================================
# PLOT 1: Odds Ratios
# ============================================
colors = ['#10b981' if or_val > 1 else '#ef4444' if or_val < 0.99 else '#6b7280' 
          for or_val in odds_ratios]

bars = ax1.bar(features, odds_ratios, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add reference line at OR = 1
ax1.axhline(y=1, color='red', linestyle='--', linewidth=2, alpha=0.7, label='No Effect (OR=1)')

# Formatting
ax1.set_ylabel('Odds Ratio', fontsize=12, fontweight='bold')
ax1.set_xlabel('Feature', fontsize=12, fontweight='bold')
ax1.set_title('Effect on Survival Odds\n(Logistic Regression)', fontsize=14, fontweight='bold', pad=20)
ax1.set_ylim(0, 2.5)
ax1.grid(axis='y', alpha=0.3)
ax1.legend(loc='upper right')

# Add value labels on bars
for i, (bar, or_val, pct) in enumerate(zip(bars, odds_ratios, percent_changes)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'OR={or_val:.3f}\n({pct:+.1f}%)',
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# ============================================
# PLOT 2: Coefficients
# ============================================
colors_coef = ['#10b981' if c > 0 else '#ef4444' if c < -0.001 else '#6b7280' 
               for c in coefficients]

bars2 = ax2.barh(features, coefficients, color=colors_coef, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add reference line at coefficient = 0
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, alpha=0.7)

# Formatting
ax2.set_xlabel('Coefficient (β)', fontsize=12, fontweight='bold')
ax2.set_title('Logistic Regression Coefficients\n(n=614, Intercept=7.609)', 
              fontsize=14, fontweight='bold', pad=20)
ax2.grid(axis='x', alpha=0.3)

# Add value labels
for i, (bar, coef) in enumerate(zip(bars2, coefficients)):
    width = bar.get_width()
    if width > 0:
        ax2.text(width + 0.02, bar.get_y() + bar.get_height()/2.,
                 f'{coef:.6f}',
                 ha='left', va='center', fontsize=9, fontweight='bold')
    else:
        ax2.text(width - 0.02, bar.get_y() + bar.get_height()/2.,
                 f'{coef:.6f}',
                 ha='right', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('smoking_survival_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================
# PLOT 3: Survival Probability by Age (Separate Figure)
# ============================================
fig, ax = plt.subplots(figsize=(12, 7))

# Generate survival probability curves
ages = np.arange(20, 81, 1)

def survival_prob(age, is_smoker, is_female):
    """Calculate survival probability using logistic regression formula"""
    log_odds = 7.6094 + (0.714646 if is_female else 0) + (-0.242709 if is_smoker else 0) - (age * 0.126916)
    return 1 / (1 + np.exp(-log_odds))

# Calculate probabilities for different groups
prob_female_nonsmoker = [survival_prob(age, False, True) * 100 for age in ages]
prob_male_nonsmoker = [survival_prob(age, False, False) * 100 for age in ages]
prob_female_smoker = [survival_prob(age, True, True) * 100 for age in ages]
prob_male_smoker = [survival_prob(age, True, False) * 100 for age in ages]

# Plot curves
ax.plot(ages, prob_female_nonsmoker, color='#10b981', linewidth=3, label='Female Non-smoker', linestyle='-')
ax.plot(ages, prob_male_nonsmoker, color='#3b82f6', linewidth=3, label='Male Non-smoker', linestyle='-')
ax.plot(ages, prob_female_smoker, color='#f59e0b', linewidth=3, label='Female Smoker', linestyle='--')
ax.plot(ages, prob_male_smoker, color='#ef4444', linewidth=3, label='Male Smoker', linestyle='--')

# Formatting
ax.set_xlabel('Age (years)', fontsize=13, fontweight='bold')
ax.set_ylabel('Survival Probability (%)', fontsize=13, fontweight='bold')
ax.set_title('Predicted Survival Probability by Age\n(After Controlling for All Factors)', 
             fontsize=15, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.set_xlim(20, 80)
ax.set_ylim(0, 100)

# Add annotation
ax.annotate('Smoking reduces survival\nacross all age groups', 
            xy=(50, 50), xytext=(60, 30),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, fontweight='bold', color='red',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('survival_probability_by_age.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================
# SUMMARY STATISTICS TABLE
# ============================================
print("\n" + "="*70)
print("SUMMARY OF FINDINGS")
print("="*70)

summary_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': coefficients,
    'Odds Ratio': odds_ratios,
    'Effect (%)': percent_changes,
    'Interpretation': [
        'Being female DOUBLES survival odds',
        'Smoking REDUCES survival odds by 22%',
        'Each year of age reduces odds by 12%',
        'Salary has virtually no effect'
    ]
})

print(summary_df.to_string(index=False))
print("\n" + "="*70)
print("✓ Smoking has a significant NEGATIVE effect on survival")
print("  even after controlling for age, gender, and salary.")
print("="*70)